In [1]:
import os
import numpy as np
from scipy.sparse import csr_matrix
import pandas as pd

## Read in and create sparse matrix from Narragansett Bay hits ##

In [11]:
diamond_file="/vortexfs1/omics/alexander/akrinos/2021-testing-eukrhythmic/"+\
            "eukrhythmic-eLife/snakemake-workflows/eukulele-snake/"+\
            "EUKulele_marmmetsp_NB_CAG_better_diatom/mets_full/diamond/NarBay_S4_CAG.fasta.transdecoder.diamond.out"

In [16]:
diamond_file="/vortexfs1/omics/alexander/akrinos/2022-euk-diversity/output/"+\
             "bidirectional-hit-dbs-modids/pep_diamond_matches/NB/hits-NarBay_S4.tsv"

In [2]:
input_tax="/vortexfs1/omics/alexander/akrinos/remodeling/EUKulele/databases/marmmetsp_better_diatom_taxonomy/tax-table.txt"

tax_level = "family"
## evaluate process of clustering with tax coherence.
tax_table = pd.read_csv(input_tax,sep="\t")
tax_table.columns = [curr.lower() for curr in tax_table.columns]

tax_table["domain"] = [str(curr).strip() for curr in tax_table.domain]
tax_table["supergroup"] = [str(curr).strip() for curr in tax_table.supergroup]
tax_table["division"] = [str(curr).strip() for curr in tax_table.division]
tax_table["class"] = [str(curr).strip() for curr in tax_table["class"]]
tax_table["order"] = [str(curr).strip() for curr in tax_table.order]
tax_table["family"] = [str(curr).strip() for curr in tax_table.family]
tax_table["genus"] = [str(curr).strip() for curr in tax_table.genus]
tax_table["species"] = [str(curr).strip() for curr in tax_table.species]

In [72]:
df = pd.read_csv(diamond_file, sep="\t", header=None,names=["qseqid","sseqid","pident","length",
                                        "mismatch","gapopen","qstart","qend","sstart","send",
                                        "evalue","bitscore"])
df = df.loc[:,["qseqid","sseqid","pident","bitscore"]]

In [73]:
a = df.sseqid.str.split("_",expand=True)
df["source_id"] = [col3 if (col1!=col1)|(col1 is None) else col1 if (col2!=col2)|(col2 is None) else\
                   col2 for col2,col1,col3 in zip(a.iloc[:,-1],a.iloc[:,-2],a.iloc[:,-3])]
df = df.merge(tax_table,how="left")

In [87]:
df.loc[df.family.isin(["Hemiaulaceae","Skeletonemataceae",
                       "Rhizosoleniaceae","Thalassiosiraceae"]),:].\
        groupby(["family"]).pident.mean().reset_index()

,family,pident
0,Hemiaulaceae,65.451708
1,Rhizosoleniaceae,60.077226
2,Skeletonemataceae,84.225632
3,Thalassiosiraceae,67.937611


In [92]:
pident_thres=50
df=df.loc[df.pident>pident_thres,:]
df.family=[str(curr).strip() for curr in df.family]
df=df.groupby(["qseqid","family"])["source_id"].count().reset_index()

In [94]:
from scipy.sparse import csr_matrix

sequence_u = list(sorted(df.qseqid.unique()))
family_u = list(sorted(df.family.unique()))

data = df['source_id'].tolist()
row = pd.Categorical(df.qseqid,categories=sequence_u).codes
col =pd.Categorical(df.family,categories=family_u).codes  #df.family.astype('category', categories=family_u).cat.codes
sparse_matrix = csr_matrix((df['source_id'].tolist(), (row, col)), shape=(len(sequence_u), len(family_u)))

In [106]:
del df

## Topic modeling approach

In [95]:
from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 30

model = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)

In [96]:
model.fit(sparse_matrix)

LatentDirichletAllocation(n_components=30, random_state=0)

In [97]:
def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
    return pd.DataFrame(topic_dict)

In [98]:
def topic_vote_dict(model, feature_names, no_top_words):
    topic_dict = {}
    topic_vote_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        print(feature_names[topic.argsort()[:-no_top_words - 1:-1]])
        topic_vote_dict[topic_idx] = feature_names[topic.argsort()[-1]]
    return topic_vote_dict

In [143]:
#topic_vote_dict(model,
#               pd.Categorical(df.family,categories=family_u)[col],
#               no_top_words)

In [100]:
no_top_words = 10
display_topics(model,
               pd.Categorical(df.family,categories=family_u)[col],
               no_top_words)

,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights,...,Topic 25 words,Topic 25 weights,Topic 26 words,Topic 26 weights,Topic 27 words,Topic 27 weights,Topic 28 words,Topic 28 weights,Topic 29 words,Topic 29 weights
0,Thoracosphaeraceae,68210.4,Chaetocerotales,26917.1,Chlorarachnida_X,173017.0,Lithodesmiaceae,45095.6,Chrysotilaceae,72946.0,...,Suessiaceae,119914.4,Thraustochytriaceae,35151.6,Chlamydomonadales_X,80543.7,Hemiaulaceae,111672.5,Chlorodendraceae,25933.2
1,Rhizosoleniaceae,38590.7,Prorocentraceae,26540.0,Picocystaceae,3589.1,Chloropicaceae,21701.0,Rosalidae,33402.6,...,Prorocentraceae,52349.6,Chloropicaceae,33250.3,Bacillariaceae,7584.9,Chlorodendraceae,83570.6,Kryptoperidiniaceae,12464.0
2,Cryptomonadales_X,36642.4,Gymnodiniaceae,2508.2,Crustomastigaceae,1962.8,Amphidomataceae,20251.0,Kareniaceae,3278.1,...,Picocystaceae,4935.3,Pseudomonadaceae,15339.7,Chloropicaceae,6492.6,Shewanellaceae,46545.0,Labyrinthulaceae,7174.0
3,Goniodomataceae,24751.8,Goniodomataceae,2131.8,Lithodesmiaceae,1962.0,Ceratiaceae,10409.5,Gymnodiniaceae,595.9,...,Prorocentraceae,3060.1,Picocystaceae,12862.8,Bacillariaceae,6368.6,Chaetocerotales,27463.6,Prorocentraceae,6108.9
4,Mamiellaceae,22802.6,Chrysotilaceae,1706.0,Crustomastigaceae,1285.8,Suessiaceae,9233.6,Goniodomataceae,564.8,...,Gymnodiniaceae,2639.8,Suessiaceae,8458.8,Picocystaceae,2947.0,Filamoebidae,21686.2,Bolidomonadaceae,2557.2
5,Isochrysidaceae,21638.2,Lithodesmiaceae,1082.2,Gymnodiniaceae,1223.4,Chlamydomonadales_X,9004.2,Suessiaceae,425.4,...,Chlorodendraceae,1653.2,Glaucocystophytes_X,6143.1,Corethraceae,2356.4,Filamoebidae,12587.0,Protocruziidae,2001.0
6,Chloropicaceae,16016.9,Chaetocerotales,1046.7,Picocystaceae,959.7,Cymatosiraceae,8240.9,Shewanellaceae,252.3,...,Thalassiosiraceae,1097.1,Chaetocerotales,5762.6,Bolidomonadaceae,1646.0,Odontellaceae,6382.5,Lithodesmiaceae,1733.0
7,Prorocentraceae,14154.1,Chloropicaceae,787.0,Chrysotilaceae,860.3,Picocystaceae,5944.2,Chaetocerotales,98.5,...,Chaetocerotales,1024.5,Gymnodiniaceae,2674.4,Chaetocerotales,1487.3,Suessiaceae,4977.4,Gonyaulacaceae,1644.1
8,Chlorodendraceae,13238.4,Chloropicaceae,682.0,Hemiaulaceae,823.4,Chaetocerotales,5322.0,Bacillariaceae,68.2,...,Kareniaceae,725.4,Chlorodendraceae,2439.4,Suessiaceae,1327.3,Favellidae,4911.1,Picocystaceae,1526.3
9,Lithodesmiaceae,12236.0,Corethraceae,575.3,Suessiaceae,486.7,Chaetocerotales,4829.6,Ceratiaceae,62.8,...,Gymnodiniaceae,672.7,Stylonematales-Group-II,2023.7,Goniodomataceae,1279.4,Chloropicaceae,4892.8,Pseudomonadaceae,1215.6


In [51]:
topic_values = model.transform(sparse_matrix)

In [56]:
len(set(topic_values.argmax(axis=1)))

30

In [54]:
topic_values.argmax(axis=1)

array([19,  7, 10, ..., 25, 25, 25])

## Affinity propagation clustering approach

In [101]:
from sklearn.cluster import AffinityPropagation
# define the model
model_apm = AffinityPropagation(damping=0.9)
# fit the model
model_apm.fit(sparse_matrix)
# assign a cluster to each example
yhat = model_apm.predict(X)
# retrieve unique clusters
clusters = unique(yhat)

MemoryError: Unable to allocate 745. GiB for an array with shape (100037022196,) and data type int64

In [47]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in globals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

                        all_df: 135.4 GiB
                            _5: 315.3 MiB
                             a: 315.3 MiB
                   set_qseqids: 136.0 MiB
                      test_ids: 34.0 MiB
                           _19:  3.2 KiB
                           _41:  1.2 KiB
                    csr_matrix:  1.0 KiB
                           _i2:  928.0 B
                           _oh:  640.0 B


In [59]:
del li

In [61]:
dir()

['In',
 'Out',
 '_',
 '_19',
 '_22',
 '_26',
 '_29',
 '_30',
 '_31',
 '_33',
 '_34',
 '_35',
 '_39',
 '_41',
 '_43',
 '_44',
 '_45',
 '_48',
 '_51',
 '_52',
 '_53',
 '_56',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i10',
 '_i11',
 '_i12',
 '_i13',
 '_i14',
 '_i15',
 '_i16',
 '_i17',
 '_i18',
 '_i19',
 '_i2',
 '_i20',
 '_i21',
 '_i22',
 '_i23',
 '_i24',
 '_i25',
 '_i26',
 '_i27',
 '_i28',
 '_i29',
 '_i3',
 '_i30',
 '_i31',
 '_i32',
 '_i33',
 '_i34',
 '_i35',
 '_i36',
 '_i37',
 '_i38',
 '_i39',
 '_i4',
 '_i40',
 '_i41',
 '_i42',
 '_i43',
 '_i44',
 '_i45',
 '_i46',
 '_i47',
 '_i48',
 '_i49',
 '_i5',
 '_i50',
 '_i51',
 '_i52',
 '_i53',
 '_i54',
 '_i55',
 '_i56',
 '_i57',
 '_i58',
 '_i59',
 '_i6',
 '_i60',
 '_i61',
 '_i7',
 '_i8',
 '_i9',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'all_df',
 'curr',
 'dir_in',
 'exit',
 'get_ipython',
 'name',
 'np',
 'os',
 'pd',
 'quit',
 'random',
 'save_suffix',


## Random Forest with AdaBoost training approach

In [4]:
dir_in="../../../output/bidirectional-hit-dbs-modids/pep_diamond_matches/small_files_larger_chunks/"
save_suffix="100est_26Dec22_allfams"

li = []
for curr in os.listdir(dir_in):
    df = pd.read_csv(os.path.join("../../scripts",'all_'+str(save_suffix)+"_"+curr+'.csv'),
                     usecols=["qseqid","sseqid","pident","bitscore","source_id",
                                                                     "source_id_q",
                                                        "family","Family","MatchedFam"],
                     dtype={"qseqid":str,"sseqid":str,
                            "pident":str,"bitscore":str,
                            "source_id":str,"source_id_q":str,"family":str,"Family":str,
                            "MatchedFam":str})
    df=df.loc[df.qseqid!=df.sseqid,:]
    df=df.loc[df.pident.astype(float) < 80,:]
    li.append(df)

all_df = pd.concat(li, axis=0, ignore_index=True)

In [5]:
import gc
gc.collect()

20

In [6]:
all_df.drop(["source_id","source_id_q","MatchedFam"],inplace=True,axis=1)

In [8]:
all_df.pident=all_df.pident.astype("float16")
all_df.bitscore=all_df.bitscore.astype("float16")

In [9]:
all_df.dtypes

qseqid       object
sseqid       object
pident      float16
bitscore    float16
family       object
Family       object
dtype: object

In [11]:
import random
random.seed(42)
set_qseqids=list(set(all_df.qseqid))
test_ids=random.sample(set_qseqids, int(0.25*len(set_qseqids)))
a = all_df.qseqid.isin(test_ids)

test_df = all_df[a]
#all_df.drop(x, inplace=True, axis=0)
train_df = all_df[~a] #all_df[~a]
#del all_df

In [12]:
all_df_filt = train_df.drop_duplicates(subset=["Family","qseqid","sseqid","pident"]).\
    loc[all_df_filt.pident >= pident,:]
all_df_filt_2 = all_df_filt.loc[all_df_filt.qseqid!=all_df_filt.sseqid,:]
counted_by_fam = all_df_filt_2.groupby(["qseqid","Family","family"])["sseqid"].count().reset_index()

num_families = tax_table.drop_duplicates(subset=["family","source_id"]).\
    groupby("family")["source_id"].count().reset_index().\
    rename({"source_id":"Num_References"},axis="columns")

full_frame = counted_by_fam.merge(num_families,how="outer",
                                  left_on="family",right_on="family").fillna(0)
full_frame["Pident"] = pident
combined_frame_train = pd.concat([combined_frame_train,full_frame])
combined_frame_train = combined_frame_train.groupby(["qseqid","Family","family"])["sseqid"].count().reset_index()

MemoryError: cannot allocate memory for array

In [ ]:
all_df_filt = test_df.drop_duplicates(subset=["Family","qseqid","sseqid","pident"]).\
        loc[all_df_filt.pident >= pident,:]
all_df_filt_2 = all_df_filt.loc[all_df_filt.qseqid!=all_df_filt.sseqid,:]
counted_by_fam = all_df_filt_2.groupby(["qseqid","Family","family"])["sseqid"].count().reset_index()

full_frame = counted_by_fam.merge(num_total,how="outer").merge(num_families,how="outer",
                                                           left_on="family",right_on="family").fillna(0)
full_frame["Pident"] = pident
combined_frame_test = pd.concat([combined_frame_test,full_frame])
combined_frame_test = combined_frame_test.groupby(["qseqid","Family","family"])["sseqid"].count().reset_index()

In [ ]:
from scipy.sparse import csr_matrix

sequence_u = list(sorted(combined_frame_train.Family.unique()))
hits_u = list(sorted(combined_frame_train.family.unique()))

data = df['source_id'].tolist()
row = pd.Categorical(combined_frame_train.Family,categories=sequence_u).codes
col =pd.Categorical(combined_frame_train.family,categories=hits_u).codes 
sparse_matrix = csr_matrix((combined_frame_train['sseqid'].tolist(), (row, col)), 
                           shape=(len(sequence_u), len(family_u)))

In [ ]:
from imblearn.ensemble import EasyEnsembleClassifier
rf = EasyEnsembleClassifier(n_estimators=10,random_state=42)
rf.fit(sparse_matrix, row);